# GeoServer ImageMosaic Configuration for MODIS COG Datasets
Using MODIS data to create a temporal ImageMosaic, stored on NASA S3 and/or NRS ObjectStore.

Requirements:
- *PostgreSQL* 16 instance running
- *GeoServer* 2.25.0+ running
  - COG Plugin (https://docs.geoserver.org/latest/en/user/community/cog/index.html)
  - GDAL installed + ImageI/O-Ext (this should be by default)
  - A workspace named 'test' is made in GeoServer
- Valid granule HTTP links
- Python environment (3.8+) with *requests* installed


## Step 1: Configuration Files Setup

### Choose parameters
Set the parameters for the layer and database. The layer_name should match the db_name to avoid potential issues.

In [ ]:
layer_name = 'modisvi_demo'
db_name = 'modisvi_demo' # Note: For a PostGIS datastore, it is not needed that the database is already present in the PostGIS instance; it will be created automatically by the ImageMosaic reader.
workspace = 'test' #! Create this workspace in GeoServer before running this script

### Initialize .properties files
Create the necessary .properties files for configuring the ImageMosaic.

In [ ]:

# indexer.properties
indexer_properties = f"""Cog=true
PropertyCollectors=TimestampFileNameExtractorSPI[timeregex](time)
TimeAttribute=time
Schema=*the_geom:Polygon,location:String,time:java.util.Date
CanBeEmpty=true
Name={layer_name}
"""
    # in this case, the CRS is automatically detected. but if you want to use BC Albers for another dataset, you have to define the MosaicCRS --> see NRSObjStore_COG_Mosaic.ipynb 

# timeregex.properties
timeregex_properties = """regex=[0-9]{4}.[0-9]{2}.[0-9]{2},format=yyyy.MM.dd
"""

# datastore.properties
datastore_properties = f'''user=postgres
port=5432
passwd=stac
url=jdbc:postgresql:{db_name}
host=localhost
database={db_name}
driver=org.postgresql.Driver
schema=public
SPI=org.geotools.data.postgis.PostgisNGDataStoreFactory
fetch\\ size=1000
max\\ connections=20
min\\ connections=5
validate\\ connections=true
Loose\\ bbox=true
Expose\\ primary\\ key=false
Max\\ open\\ prepared\\ statements=50
preparedStatements=false
Estimated\\ extends=false
Connection\\ timeout=20
'''



import zipfile
import os


### Create a zip archive with the configuration files
Bundle the .properties files into a single ZIP archive for uploading to GeoServer.

In [ ]:
zip_name = f'{layer_name}.zip'

with zipfile.ZipFile(zip_name, 'w') as zipf:
    with zipf.open('indexer.properties', 'w') as f:
        f.write(indexer_properties.encode())
    with zipf.open('timeregex.properties', 'w') as f:
        f.write(timeregex_properties.encode())
    with zipf.open('datastore.properties', 'w') as f:
        f.write(datastore_properties.encode())


## Step 2: REST API Calls to GeoServer
Use the GeoServer REST API to configure the ImageMosaic. This involves creating an empty ImageMosaic, providing a prototype dataset, initializing the store, configuring the coverage, and adding more granules.

In [ ]:
import requests

# Base GeoServer URL and credentials
base_url = "http://localhost:8080/geoserver"
store_name = layer_name
user = "admin"
password = "geoserver"

# 1. Create an empty ImageMosaic
url = f"{base_url}/rest/workspaces/{workspace}/coveragestores/{store_name}/file.imagemosaic?configure=none"
headers = {"Content-type": "application/zip"}

with open(zip_name, 'rb') as file:
    response = requests.put(url, auth=(user, password), headers=headers, data=file)
print(f"Create empty ImageMosaic response: {response.status_code}")


In [ ]:

# 2. Provide a prototype dataset
url = f"{base_url}/rest/workspaces/{workspace}/coveragestores/{store_name}/remote.imagemosaic"
headers = {"Content-type": "text/plain"}
data = "https://modis-vi-nasa.s3-us-west-2.amazonaws.com/MOD13A1.006/2018.01.01.tif" # NASA host
# data = "https://nrs.objectstore.gov.bc.ca/cloudgistest/test/cog_out/2018.01.01.tif" # NRS ObjectStore host
response = requests.post(url, auth=(user, password), headers=headers, data=data)
print(f"Provide prototype dataset response: {response.status_code}")


In [ ]:

# 3. Initialize the store (Listing available coverages)
url = f"{base_url}/rest/workspaces/{workspace}/coveragestores/{store_name}/coverages.xml?list=all"
response = requests.get(url, auth=(user, password))
print(f"Initialize the store response: {response.status_code}")
print(response.text)


The output should be:
```
<list>
  <coverageName>modisvi_demo</coverageName>
</list>
```

In [ ]:

# 4. Configure the coverage
coverage_xml = f"""
<coverage>
  <name>{layer_name}</name>
  <nativeName>{layer_name}</nativeName>
  <enabled>true</enabled>
  <metadata>
    <entry key="time">
      <dimensionInfo>
        <enabled>true</enabled>
        <presentation>LIST</presentation>
        <units>ISO8601</units>
        <defaultValue>
          <strategy>MAXIMUM</strategy>
        </defaultValue>
      </dimensionInfo>
    </entry>
  </metadata>
</coverage>
"""
url = f"{base_url}/rest/workspaces/{workspace}/coveragestores/{store_name}/coverages"
headers = {"Content-type": "text/xml"}
response = requests.post(url, auth=(user, password), headers=headers, data=coverage_xml)
print(f"Configure the coverage response: {response.status_code}")


In [ ]:

# 5. Add more granules
url = f"{base_url}/rest/workspaces/{workspace}/coveragestores/{store_name}/remote.imagemosaic"
data = "https://modis-vi-nasa.s3-us-west-2.amazonaws.com/MOD13A1.006/2018.01.17.tif"
response = requests.post(url, auth=(user, password), headers=headers, data=data)
print(f"Add more granules response: {response.status_code}")


In [ ]:

# 6. Setting Style
ndvi_sld = """
<StyledLayerDescriptor xmlns="http://www.opengis.net/sld" xmlns:sld="http://www.opengis.net/sld" xmlns:gml="http://www.opengis.net/gml" xmlns:ogc="http://www.opengis.net/ogc" version="1.0.0">
  <NamedLayer>
    <Name>Default Styler</Name>
    <UserStyle>
      <Name>ndvi</Name>
      <Title>ndvi</Title>
      <FeatureTypeStyle>
        <Name>name</Name>
        <Rule>
          <RasterSymbolizer>
            <ChannelSelection>
              <GrayChannel>
                <SourceChannelName>1</SourceChannelName>
              </GrayChannel>
            </ChannelSelection>
            <ColorMap>
              <ColorMapEntry color="#000000" quantity="-1"/>
              <ColorMapEntry color="#0000ff" quantity="-0.75"/>
              <ColorMapEntry color="#ff00ff" quantity="-0.25"/>
              <ColorMapEntry color="#ff0000" quantity="0"/>
              <ColorMapEntry color="#ffff00" quantity="0.5"/>
              <ColorMapEntry color="#00ff00" quantity="1"/>
            </ColorMap>
            <ContrastEnhancement/>
          </RasterSymbolizer>
        </Rule>
      </FeatureTypeStyle>
    </UserStyle>
  </NamedLayer>
</StyledLayerDescriptor>
"""
with open('ndvi.sld', 'w') as f:
    f.write(ndvi_sld)

url = f"{base_url}/rest/styles"
headers = {"Content-type": "application/vnd.ogc.sld+xml"}

with open('ndvi.sld', 'rb') as file:
    response = requests.post(url, auth=(user, password), headers=headers, data=file)
print(f"Create the style response: {response.status_code}")


In [ ]:

# 7. Set the style as default style of the layer
url = f"{base_url}/rest/layers/{store_name}.xml"
data = "<layer><defaultStyle><name>ndvi</name></defaultStyle></layer>"
headers = {"Content-type": "text/xml"}
response = requests.put(url, auth=(user, password), headers=headers, data=data)
print(f"Set default style response: {response.status_code}")


In [ ]:
# Verify that the ImageMosaic layer exists and get details
url = f"{base_url}/rest/layers/{store_name}.xml"
response = requests.get(url, auth=(user, password))

if response.status_code == 200:
    print("ImageMosaic layer exists.")
    # Print the details of the layer
    print("Layer details:")
    print(response.text)
else:
    print(f"Failed to find the ImageMosaic layer. Status code: {response.status_code}")

# Get details about the granules in the coverage
coverage_url = f"{base_url}/rest/workspaces/{workspace}/coveragestores/{store_name}/coverages.xml"
coverage_response = requests.get(coverage_url, auth=(user, password))

if coverage_response.status_code == 200:
    print("Coverage details:")
    print(coverage_response.text)
else:
    print(f"Failed to get coverage details. Status code: {coverage_response.status_code}")

Another issue appeared when trying to preview the file: the 'time' column of the dataset was empty. I manually updated it and it worked... 
